In [2]:
import pandas as pd
import numpy as np
import calendar

In [ ]:
input_file = 'file_path'
output_file = 'file_path_new'

# Read the data from the Excel file
df = pd.read_excel(input_file)

# Extract year, month, and quarter from the 'date' column
df['year'] = df['Date'].str.extract(r'(\d{4})')
df['month'] = df['Date'].str.extract(r'M(\d{2})').astype(int)
df['quarter'] = df['month'].apply(lambda m: (m-1) // 3 + 1)

# Save the modified data back to a new Excel file
df.to_excel(output_file, index=False, engine='openpyxl')

print("Data has been separated and saved to '{}'.".format(output_file))


In [ ]:
# Function to handle null and missing values
def handle_missing_values(df):
    # Iterate over the columns and handle null values
    for col in df.columns[2:]:
        # Calculate the mean and median
        mean_val = df[col].mean()
        median_val = df[col].median()

        # Check for outliers using the interquartile range (IQR)
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Replace null values based on outliers, skewness, and missing data impact
        if lower_bound <= median_val <= upper_bound:
            # No significant outliers, use the mean
            df[col].fillna(mean_val, inplace=True)
        else:
            # Outliers present, use the median
            df[col].fillna(median_val, inplace=True)

    return df

# Load the existing CPI_updated.xlsx file
file_path = 'file_path'
excel_file = pd.ExcelFile(file_path)

# Get the first sheet name
sheet_name = excel_file.sheet_names[0]

# Read the first sheet into a DataFrame
df_monthly = excel_file.parse(sheet_name)

# Handle null and missing values
df_monthly = handle_missing_values(df_monthly)

# Save the updated data back to the first sheet of the existing file
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df_monthly.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Null and missing values replaced in the '{sheet_name}' sheet of {file_path}.")

In [11]:
# Read the Excel file
df = pd.read_excel('file_path')

# Get the country columns (excluding the first three columns)
country_columns = df.columns[3:]

# Reshape the DataFrame to have 'Country' as a column and other columns as values
df = df.melt(id_vars=['year', 'quarter', 'month'], value_vars=country_columns, var_name='Country', value_name='Data')

# Convert the file format to CSV and save it
df.to_csv('file_path.csv', index=False)